<a href="https://colab.research.google.com/github/wangbxj1234/offset_pt_try/blob/main/load_data_byfps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import glob
import h5py
import numpy as np
from torch.utils.data import Dataset
import os
import torch

def farthest_point_sample(xyz, npoint):
    """
    Input:
        xyz: pointcloud data, [B, N, 3]
        npoint: number of samples
    Return:
        centroids: sampled pointcloud index, [B, npoint]
    """
    device = xyz.device
    B, N, C = xyz.shape#############9840, 2048, 3
    centroids = torch.zeros(B, npoint, dtype=torch.long).to(device)#(9840, 1024)
    distance = torch.ones(B, N).to(device) * 1e10        ###########(9840, 2048)
    farthest = torch.randint(0, N, (B,), dtype=torch.long).to(device)######low:0,high:2048,  (9840,1)
    batch_indices = torch.arange(B, dtype=torch.long).to(device)########### arange: to generate (0,1,2,.....,2047)
    for i in range(npoint):
        centroids[:, i] = farthest##################[9840,1]
        centroid = xyz[batch_indices, farthest, :].view(B, 1, 3) ######(9840,2048,3)
        dist = torch.sum((xyz - centroid) ** 2, -1)##################(9840,2048)
        distance = torch.min(distance, dist)###############NO LARGER THAN 1e10
        farthest = torch.max(distance, -1)[1]############## 取最远距离的那个点 得到坐标
    return centroids


def index_points(points, idx):
    """
    Input:
        points: input points data, [B, N, C]
        idx: sample index data, [B, S, [K]]
    Return:
        new_points:, indexed points data, [B, S, [K], C]
    """
    raw_size = idx.size()
    idx = idx.reshape(raw_size[0], -1)
    res = torch.gather(points, 1, idx[..., None].expand(-1, -1, points.size(-1)))
    return res.reshape(*raw_size, -1)
def load_data(partition):
    all_data = []
    all_label = []
    for h5_name in glob.glob('/content/drive/MyDrive/PAConv-main/modelnet40_ply_hdf5_2048/ply_data_%s*.h5' % partition):
        f = h5py.File(h5_name, mode='r')
        data = f['data'][:].astype('float32')
        label = f['label'][:].astype('int64')
        f.close()
 ######### 
        data = torch.Tensor(data) 
        print(data.shape)     
        fps_idx_single = farthest_point_sample(data, 1024)
        data_1 = index_points(data, fps_idx_single)########### 按照index 返回 3维点。
        data=data_1.data.numpy()
        all_data.append(data)
 #########
        all_label.append(label)
        #
    all_data = np.concatenate(all_data, axis=0)
    all_label = np.concatenate(all_label, axis=0)
    print('all_data.shape:',all_data.shape)
    return all_data, all_label



In [8]:
partition='train'
d_1,l_1=load_data(partition)

torch.Size([1648, 2048, 3])
torch.Size([2048, 2048, 3])
torch.Size([2048, 2048, 3])
torch.Size([2048, 2048, 3])
torch.Size([2048, 2048, 3])
all_data.shape: (9840, 1024, 3)
